In [24]:
#from pydoc import cli

import numpy as np
#import incoming as inc
import matplotlib.pyplot as plt
import pandas as pd
from alpaca.data.historical import StockHistoricalDataClient, OptionHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, StockQuotesRequest, OptionTradesRequest, OptionBarsRequest, OptionLatestQuoteRequest
OptionBarsRequest
from alpaca.data.timeframe import TimeFrame ; import cudf as cdf 
from datetime import datetime, timezone, date
import polars as pl
from alpaca.data.live import StockDataStream, OptionDataStream
from account import API_KEY , API_SECRET
from collections import  deque



In [3]:
# alpaca_data_client.py


class AlpacaDataClient:
    def __init__(self, api_key: str, api_secret: str):
        self.stock_hist = StockHistoricalDataClient(api_key, api_secret)
        self.option_hist = OptionHistoricalDataClient(api_key, api_secret)

        self.stock_stream = StockDataStream(api_key, api_secret)
        self.option_stream = OptionDataStream(api_key, api_secret)

    # =========================
    # HISTORICAL — STOCKS
    # =========================
    def stock_bars(self, symbols, timeframe, start=None, end=None, limit=None):
        req = StockBarsRequest(
            symbol_or_symbols=symbols,
            timeframe=timeframe,
            start=start,
            end=end,
            limit=limit
        )
        return self.stock_hist.get_stock_bars(req)

    def stock_trades(self, symbols, timeframe, start=None, end=None, limit=None):
        req = StockQuotesRequest(
            symbol_or_symbols=symbols,
            timeframe=timeframe,

            start=start,
            end=end,
            limit=limit
        )
        return self.stock_hist.get_stock_trades(req)

    def stock_quotes(self, symbols, start=None, end=None
, limit=None):
        req = StockQuotesRequest(
            symbol_or_symbols=symbols,
            start=start,
            end=end,
            limit=limit
        )
        return self.stock_hist.get_stock_quotes(req)

    # =========================
    # HISTORICAL — OPTIONS
    # =========================
    def option_bars(self, symbols, timeframe, start=None, end=None, limit=None):
        req = OptionBarsRequest(
            symbol_or_symbols=symbols,
            timeframe=timeframe,
            start=start,
            end=end,
            limit=limit
        )
        return self.option_hist.get_option_bars(req)

    def option_trades(self, symbols, start=None, end=None, limit=None):
        req = OptionTradesRequest(
            symbol_or_symbols=symbols,
            start=start,
            end=end,
            limit=limit
        )
        return self.option_hist.get_option_trades(req)

    def option_quotes(self, symbols, start=None, end=None, limit=None):
        req = OptionTradesRequest(
            symbol_or_symbols=symbols,
            start=start,
            end=end,
            limit=limit
        )
        return self.option_hist.get_option_latest_quote(req)

    # =========================
    # STREAMING — STOCKS
    # =========================
    def stream_stock_bars(self, symbols, handler):
        self.stock_stream.subscribe_bars(handler, *symbols)

    def stream_stock_trades(self, symbols, handler):
        self.stock_stream.subscribe_trades(handler, *symbols)

    def stream_stock_quotes(self, symbols, handler):
        self.stock_stream.subscribe_quotes(handler, *symbols)

    # =========================
    # STREAMING — OPTIONS
    # =========================
    def stream_option_trades(self, symbols, handler):
        self.option_stream.subscribe_trades(handler, *symbols)

    def stream_option_quotes(self, symbols, handler):
        self.option_stream.subscribe_quotes(handler, *symbols)

    # =========================
    # RUN STREAMS
    # =========================
    async def run_stock_stream(self):
        await self.stock_stream.run()

    async def run_option_stream(self):
        await self.option_stream.run()


In [4]:

yymmdd = date.today().strftime("%y%m%d")  # "260204"
yymmdd

'260208'

Calls 


ReQuests 

In [ ]:
list = ["SYM", "INTC", "NVDA", "ARM", "AMD"]
start_date = datetime(2026,1,25)
end_date = datetime.now()
list[0]

def MakeOptionsym(sym:str, type: str, date : datetime, strike):
    date =  date.strftime("%y%m%d")

    strike = int(round(strike * 1000))

    pad = 8 - len(str(strike))
    pad = str(0).zfill(pad)
    i = sym+str(date)+type+str(pad)+str(strike)
    return i

z = MakeOptionsym(list[1],"C",datetime(2026,1,30), 75)
print(z)



INTC260130C00075000


In [ ]:
client  = AlpacaDataClient(API_KEY, API_SECRET) 
S_Quotes = client.stock_quotes(symbols=list[0],   start=start_date, end=datetime.now()).df
S_Bars = client.stock_bars(symbols=list[0],timeframe=TimeFrame.Minute, start=start_date, end=datetime.today()).df
S_Trades = client.stock_trades(symbols=list[0],timeframe=TimeFrame.Hour,start=datetime(2026, 1, 30), end=datetime.today()).df
# remeber old time and call
#  and new time and add the data frame btw those times 


In [7]:

#Options 
#O_Quotes = client.option_quotes(z, start=start_date, end=datetime.today())
O_Bars = client.option_bars(z,timeframe=TimeFrame.Hour,start= start_date, end=datetime.today())
O_Trades = client.option_trades(z, start=start_date, end=datetime.today())

In [ ]:
import time
from datetime import datetime

# purpose was to update data each frame 


def Update():
    
    t2 , t1 = datetime.now()
    S_Quotes = client.stock_quotes(symbols=list[0],  
             start=t2, end=t1).df

    d = deque()
    d.append(S_Quotes)

    t2 = datetime.now()
    print(t2 - t1)
    
Update()

UnboundLocalError: cannot access local variable 't2' where it is not associated with a value

In [7]:
S_Quotes

bid_price  bid_size bid_exchange  \
symbol timestamp                                                            
SYM    2026-01-26 09:00:00.012017+00:00       48.0     100.0            K   
       2026-01-26 09:00:00.012036+00:00       52.0     100.0            K   
       2026-01-26 09:00:00.012053+00:00       55.7     100.0            K   
       2026-01-26 09:00:00.012105+00:00       56.0     100.0            K   
       2026-01-26 09:00:00.012172+00:00       58.0     100.0            K   
...                                            ...       ...          ...   
       2026-02-06 11:28:26.645325+00:00       55.5    2000.0            Q   
       2026-02-06 11:28:38.001430+00:00       55.5     200.0            K   
       2026-02-06 11:30:35.567990+00:00       55.8     100.0            Q   
       2026-02-06 11:30:35.568042+00:00       55.5     200.0            K   
       2026-02-06 11:33:40.968956+00:00       55.5     300.0            K   

                                         ask_price  ask_size ask_exchange  \
symbol timestamp                                                            
SYM    2026-01-26 09:00:00.012017+00:00       0.00       0.0                
       2026-01-26 09:00:00.012036+00:00       0.00       0.0                
       2026-01-26 09:00:00.012053+00:00      64.99     300.0            K   
       2026-01-26 09:00:00.012105+00:00      64.99     300.0            K   
       2026-01-26 09:00:00.012172+00:00      64.99     300.0            K   
...                                            ...       ...          ...   
       2026-02-06 11:28:26.645325+00:00      56.00     100.0            P   
       2026-02-06 11:28:38.001430+00:00      56.00     100.0            P   
       2026-02-06 11:30:35.567990+00:00      56.00     100.0            P   
       2026-02-06 11:30:35.568042+00:00      56.00     100.0            P   
       2026-02-06 11:33:40.968956+00:00      56.00     100.0            P   

                                        conditions tape  
symbol timestamp                                         
SYM    2026-01-26 09:00:00.012017+00:00        [Y]    C  
       2026-01-26 09:00:00.012036+00:00        [Y]    C  
       2026-01-26 09:00:00.012053+00:00        [R]    C  
       2026-01-26 09:00:00.012105+00:00        [R]    C  
       2026-01-26 09:00:00.012172+00:00        [R]    C  
...                                            ...  ...  
       2026-02-06 11:28:26.645325+00:00        [R]    C  
       2026-02-06 11:28:38.001430+00:00        [R]    C  
       2026-02-06 11:30:35.567990+00:00        [R]    C  
       2026-02-06 11:30:35.568042+00:00        [R]    C  
       2026-02-06 11:33:40.968956+00:00        [R]    C  

[236027 rows x 8 columns]

In [8]:
#sum up all recurring size for a specific price: highlights where trades has occured more often 
from cudf import dtype

bid_x_size = pd.DataFrame(S_Quotes).reset_index().groupby('bid_price')['bid_size'].sum()
bid_x_size = bid_x_size.reset_index()

ask_x_size = (
    pd.DataFrame(S_Quotes)
      .reset_index()
      .groupby('ask_price')['ask_size']
      .sum()
      .reset_index()
)




In [9]:
import pyqtgraph as pg 

offset = 0.5  # half a tick, adjust to your market

app = pg.mkQApp()
win = pg.GraphicsLayoutWidget(show=True)
p1 = win.addPlot(title = "bids")
p1.showGrid(x = True, y = True)
x1 = pg.BarGraphItem(x=bid_x_size.bid_price - offset, 
                     height=bid_x_size.bid_size, width = 0.09, 
                     size=0.3, brush = pg.mkBrush(255,255,0))
p1.addItem(x1)

p1.setLabel('bottom', 'Price')
p1.setLabel('left', 'Bid Size')

win.nextRow()

p2 = win.addPlot(title = "asks")
p2.showGrid(x = True, y = True)

x2 = pg.BarGraphItem(x=ask_x_size.ask_price, 
                     height=ask_x_size.ask_size, width = 0.09, 
                     size=0.3, brush = pg.mkBrush(0,255,255))




p2.addItem(x2)
app.exec()


Gtk-Message: 11:34:46.363: Failed to load module "appmenu-gtk-module"


0

In [10]:
class Aggregations:
    def __init__(self, x: pd.DataFrame):
    
        pass 
    # Aggregations 
    


In [11]:
S_Bars['open'].diff()

symbol  timestamp                
SYM     2026-01-26 09:00:00+00:00     NaN
        2026-01-26 09:01:00+00:00   -0.10
        2026-01-26 09:03:00+00:00    0.15
        2026-01-26 10:21:00+00:00   -0.79
        2026-01-26 12:05:00+00:00    0.41
                                     ... 
        2026-02-06 11:01:00+00:00    0.01
        2026-02-06 11:02:00+00:00    0.27
        2026-02-06 11:07:00+00:00    0.79
        2026-02-06 11:25:00+00:00   -0.26
        2026-02-06 11:30:00+00:00    0.09
Name: open, Length: 4221, dtype: float64

In [8]:
import numpy as np
from collections import deque
import pyqtgraph as pg
from PyQt6 import QtCore

app = pg.mkQApp("RT Scatter")

win = pg.GraphicsLayoutWidget(title="Real-time Scatter")
p = win.addPlot()
p.showGrid(x=True, y=True)

x = np.random.normal(1,3,100)
x2 = np.random.normal(1,3,100)
y = np.random.normal(1,3,100)


l = []
def update():

    print("hello")
  

        
        
 

timer = QtCore.QTimer()
timer.timeout.connect(update)
timer.start(600)   # ~60 FPS

win.show()
pg.exec()


hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


0

In [13]:
print(type(z))

<class 'str'>
